In [1]:
import os
import sys
import json
from collections import Counter
from scipy.integrate import solve_ivp
from scipy.stats import ttest_rel
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from math import pi
import pickle
import matplotlib.pyplot as plt
from packages import data_container
from packages.data_container import Data
from packages.helper import play_trajs, rotate, sp2a, v2sp, dist, psi, beta, d_theta, d_psi, sp2v, dist, \
    traj_speed, min_sep
from packages.ode_simulator import ODESimulator
from packages.models import cohen_avoid_heading
from sklearn.metrics import mean_squared_error
from fitting import Bai_movObst1, Cohen_movObst1, Cohen_movObst2
from parameters import approaches, avoidances

# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Cohen_movObst1_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)


In [ ]:
'''Simulation with var0'''
%matplotlib qt
Hz = 100
xg0, yg0, xo0, yo0, x0, y0 = 0, 10, 5, 5, 0, 0
vxo0, vyo0, vx0, vy0 = -1, 0, 0, 0.9
s0, phi0 = v2sp([vx0, vy0])
a0 = dphi0 = ds0 = 0
w = 0.1
# xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, a, phi, s, dphi, ds = var0
var0 = [xg0, yg0, xo0, yo0, vxo0, vyo0, x0, y0, vx0, vy0, a0, phi0, s0, dphi0, ds0, w]
fajen_approach['ps'] = 1
fajen_approach2['ps'] = 1
cohen_avoid2['ps'] = 1
models = [fajen_approach2, cohen_avoid3]
dw = [0.1] * 1 * Hz + [0] * 9 * Hz
args = {'w_goal': 0.1, 'w_obst': 0.1, 'dw': 0}
sim = ODESimulator(Hz=Hz, models=models, args=args, ref=[0, 1])
sim.simulate(var0, total_time=10)
sim.play()

In [ ]:
'''Simulate one trial approach'''
############
i_trial = 0
approach = approaches['Bai_movObst1b']['fajen_approach']['differential_evolution'][0]
avoid = None
############
%matplotlib qt
models = [approach]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='stimuli_onset', t_end='stimuli_out', ps='trial')
print(sim.test('p_dist'))
sim.play(obst=False)

### Cohen_movObst1

In [3]:
'''Simulate one trial avoidance'''
############
i_trial = 0
approach = approaches['Fajen_steer1a']['fajen_approach']['differential_evolution'][-1]
# avoid = avoidances['Cohen_movObst1']['cohen_avoid']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst1']['cohen_avoid_heading']['differential_evolution'][-1]
# avoid = {'name': 'cohen_avoid_heading', 'b1': 2.1260796879288923, 'k1': 673.0529452234115, 'c5': 14.968106833780706, 'c6': 1.0453773247966653}

############
%matplotlib qt
models = [approach, avoid]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='obst_onset', t_end='obst_out', ps='trial')
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
print(sim.test('p_dist'))
sim.play(title=title)

Loading finished
Simulated 1 trials in 0:00:00
0.10210607222848386


In [2]:
'''Simuate all trials model 1'''
############
# subject = 1
approach = approaches['Fajen_steer1a']['fajen_approach']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst1']['cohen_avoid']['differential_evolution'][-1]
############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim1, trials1 = Cohen_movObst1(subjects)
sim1.reset()
sim1.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim1.simulate_all(trials1, t_start='obst_onset', t_end='obst_out', ps='trial')
print(sim1.test('p_dist'))
print(sim1.test('order_accuracy'))
print(sim1.test('phi_RMSE'))
print(sim1.test('s_RMSE'))

Loading finished
Simulated 995 trials in 0:00:32
0.19169562985664634
0.8391959798994975
0.1833723649559832
0.11994164583576297


In [3]:
'''Simuate all trials model 2'''
############
# subject = 1
approach = approaches['Fajen_steer1a']['fajen_approach']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst1']['cohen_avoid_heading']['differential_evolution'][-1]

############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim2, trials2 = Cohen_movObst1(subjects)
sim2.reset()
sim2.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim2.simulate_all(trials2, t_start='obst_onset', t_end='obst_out', ps='trial')
print(sim2.test('p_dist'))
print(sim2.test('order_accuracy'))
print(sim2.test('phi_RMSE'))
print(sim2.test('s_RMSE'))

Loading finished
Simulated 995 trials in 0:00:28
0.19450744826397065
0.84321608040201
0.1875837785131185
0.11982798351664611


In [4]:
'''Compare two simulations'''
%matplotlib qt
e1 = sim1.test('p_dist', all_errors=True)
e2 = sim2.test('p_dist', all_errors=True)
plt.plot(e1, label=sim1.models[1]['name'])
plt.plot(e2, label=sim2.models[1]['name'])
plt.legend()
plt.xlabel('trial#')
plt.ylabel('distance error (m)')

Text(0, 0.5, 'distance error (m)')

In [6]:
'''Animate certain trials'''
sim = sim2
i_trial = 367
sim.play(i_trial=i_trial)
sim.test('p_dist', i_trials=[i_trial])

0.32983936132249364

In [25]:
'''Check pass order accuracy of model1 and model2 when model1 is worse'''
acc1 = 0
acc2 = 0
total = 0
for i in np.where(np.array(e1) > np.array(e2))[0]:
    j = sim1.i_trials[i]
    pred_order1 = sim1.pass_order_pred[i]
    pred_order2 = sim2.pass_order_pred[i]
    true_order = data.info['pass_order'][j]
    acc1 += 1 if pred_order1 == true_order else 0
    acc2 += 1 if pred_order2 == true_order else 0
    total += 1
print(acc1, acc2, acc1/total, acc2/total)

328 338 0.6876310272536688 0.7085953878406709


In [26]:
'''Check the error when model1 is worse'''
ee1 = []
ee2 = []
for e1i, e2i in zip(e1, e2):
    if e1i > e2i:
        ee1.append(e1i)
    elif e1i < e2i:
        ee2.append(e1i)
print(np.mean(ee1), np.mean(ee2))

0.2455773518229158 0.1420786773897607


In [21]:
'''Check the conditions of large errors'''
cons = []
for i in np.where(np.array(e1) > 0.3)[0]:
    j = sim1.i_trials[i]
    speed = data.info['obst_speed'][j]
    angle = data.info['obst_angle'][j]
    cons.append((angle, speed))
c = Counter(cons)
print(c)

Counter({(70, 0.6): 18, (-70, 0.4): 15, (-90, 0.4): 14, (-110, 0.4): 14, (-70, 0.6): 13, (70, 0.8): 11, (90, 0.4): 11, (110, 0.6): 11, (110, 0.4): 10, (70, 0.4): 10, (90, 0.6): 10, (-90, 0.6): 9, (-110, 0.6): 7, (90, 0.8): 7, (-70, 0.8): 5, (110, 0.8): 5, (-90, 0.8): 3, (-110, 0.8): 1})


In [ ]:
'''BIC'''
sim = sim1
k = 8
n = len(sim.p_pred)
MSE = sim.test('p_MSE')*2
BIC = n * np.log(MSE) + k * np.log(n)
print(f'The BIC of model {sim.models[1]["name"]} is {BIC}, MSE is {MSE}, k is {k}, n is {n}')

In [ ]:
'''Find trials in certain range of error'''
sim = sim1
errors = sim.test('p_dist', all_errors=True)
print(f'smallest error {np.argmin(errors)}')
for i, err in enumerate(errors):
    if 0.189 < err < 0.19:
        print(i)

In [ ]:
'''Play simulations'''
sim = sim1
i_trial = 897
print(sim.test('p_dist', i_trials=[i_trial]))
sim.play(i_trial=i_trial, save=True)


In [ ]:
'''Check simulation length'''
ls = []
for i in range(len(sim.p_pred)):
    ls.append(len(sim.p_pred[i]))
np.mean(ls)/30

In [ ]:
'''Speed and heading errors correct vs incorrect trials'''
from sklearn.metrics import mean_squared_error
sim = sim1
phi_MSEs = []
speed_MSEs = []
trials = []
for i in range(len(sim.v_pred)):
    j = sim.i_trials[i]
#     if sim.pass_order_pred[i] == sim.data.info['pass_order'][j]:
    v_subj = np.gradient(sim.p_subj[i], axis=0) * sim.data.Hz
    v_pred = sim.v_pred[i]
    s0, phi0 = v2sp(v_subj)
    s1, phi1 = v2sp(v_pred)
    speed_MSEs.append(mean_squared_error(s0, s1))
    phi_MSEs.append(mean_squared_error(phi0, phi1))
    trials.append(i)
print(sim.models)
print(f'{len(phi_MSEs)} trials in total')
print(f'RMSE on heading is {np.sqrt(np.mean(phi_MSEs))}')
print(f'RMSE on speed is {np.sqrt(np.mean(speed_MSEs))}')
print(f"Distance is {sim.test('p_dist', i_trials=trials)}")

In [ ]:
sim2.test('p_dist')

In [ ]:
'''Minimum Passing Distance'''
%matplotlib qt
sim = sim2
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title('Signed predicted minimum passing distance (SMPD)')
ax.set_ylabel('SMPD (m)')
ax.set_xlabel('normalized time (%)')
ax.set_ylim((-2, 2))
for j, i in enumerate(sim.i_trials):
    t0 = data.info['obst_onset'][i]
    t1 = data.info['obst_out'][i]
    p0 = sim.p_pred[j]
    p1 = data.info['p_obst'][i][t0:t1]
    v0 = sim.v_pred[j]
    v1 = data.info['v_obst'][i][t0:t1]
    t = np.linspace(0, 100, len(p0))
    smpd = []
    for _p0, _p1, _v0, _v1 in zip(p0, p1, v0, v1):        
        smpd.append(min_sep(_p0, _p1, _v0, _v1)[0])
    ax.plot(t, smpd, 'k', linewidth=0.1, alpha=0.5)

In [ ]:
print(len(p0), len(p1), len(sim.i_trials))

In [ ]:
'''Compute pred_order'''
s = sim
orders = []
for i in range(len(s.p_pred)):
    j = s.i_trials[i]
    true_order = s.data.info['pass_order'][j]
    p0 = s.p_pred[i][-1]
    v0 = s.v_pred[i][-1]
    p1 = s.p_obst[i][-1]
    _beta = beta(p0, p1, v0)
    angle = s.data.info['obst_angle'][j]
    pred_order = np.sign(_beta * -angle)
    orders.append(true_order == pred_order)
print(np.mean(orders))

In [ ]:
sim.test('order_accuracy')

In [ ]:
'''t-test between two models'''
err1 = sim.test('p_dist', all_errors=True)
err2 = sim2.test('p_dist', all_errors=True)
print(ttest_rel(err1, err2))
print(f'sd1 = {np.std(err1)}, sd2 = {np.std(err2)}')
print(f'df = {len(err1) - 1}')

In [ ]:
'''Show one trial from batch simulations'''
######## index in simulated trials only
i = 83
########
i_trial = sim.i_trials[i]
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.play(i, title=title, save=False)

# When beta and dpsi has the same sign it means pass in front, otherwise it means pass from behind
pass_order = sim.data.info['pass_order'][i]
pred = sim.pass_order_pred[i]
print('pass order ', pass_order, 'prediction ', pred)
print(f"err is {sim.test('p_dist', i_trial=i)}")

In [ ]:
'''plot true vs predicted speed by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0.5, 2)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    s_subj, s_pred = traj_speed(subj, Hz), traj_speed(pred, Hz)
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Speed
    axes[(abs(angle), speed)].plot(t, s_subj, 'r')
    axes[(abs(angle), speed)].plot(t, s_pred, 'b')


In [ ]:
'''plot true vs predicted heading by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0, 1.5)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    h_subj, h_pred = v2sp(np.gradient(subj, axis=0) * Hz)[1], v2sp(np.gradient(pred, axis=0) * Hz)[1]
    if h_subj[0] < 0:
        h_subj += pi
        h_pred += pi
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Heading
    axes[(abs(angle), speed)].plot(t, h_subj, 'r')
    axes[(abs(angle), speed)].plot(t, h_pred, 'b')
    if abs(angle) == 112.5 and speed == 1.2:
        print(i, i_trial)


In [8]:
'''Cross validation error'''
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Results', 'Bai_movObst1_cross_validation_cohen_avoid3_with_approach_trained_on_Bai_movObst1b_all_threshold_onset'))
filenames = [os.path.join(path, name) for name in os.listdir(path) if name[-3:] == 'txt']
errors = {}
for filename in filenames:
    with open(filename, 'rb') as f:
        best = 0
        e_min = float('inf')
        if filename[-6] == '_' or filename[-6] == '-':
            subj_id = int(filename[-5])
        else:
            subj_id = int(filename[-6:-4])
        for i, line in enumerate(f):
            if i > 10:
                try:
                    err = str(line).split("\\t")[2][:10]
                    if err[0] == '0':
                        err = float(err)
                        if err < e_min:
                            e_min = err
                            best = str(line).replace("\\", "")
                except:
                    pass
    try:
#         models = json.loads(''.join(best.split('t')[1] + best.split('t')[2]).replace("\'", "\"")) # If contains acceleration model
        models = json.loads(best.split('t')[1].replace("\'", "\"")) # If not contains acceleration model
        subjects = [int(subj_id)]
        sim, trials = Bai_movObst1(subjects)
        sim.reset()
        sim.models = models
        sim.simulate_all(trials, t_start='threshold_onset', t_end='obst_out', ps='trial')
        errors[subj_id] = sim.test('p_dist', all_errors=True)
        print(f'{subj_id} finished')
    except Exception as e:
        print(e)

Loading finished
Loading finished
[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 18, 19, 24, 25, 26, 27, 28, 29, 32, 33, 36, 37, 38, 39, 42, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 68, 69, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 84, 85, 86, 89, 90, 91, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 110, 111, 112, 114, 116, 117, 119, 120, 121, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 158]
Simulated 120 trials in 0:00:02
0 finished
Loading finished
Loading finished
[160, 162, 163, 164, 165, 166, 167, 168, 169, 171, 172, 173, 175, 177, 178, 179, 181, 183, 184, 186, 187, 189, 190, 192, 193, 195, 196, 197, 200, 201, 203, 204, 205, 206, 207, 209, 210, 212, 213, 214, 215, 216, 218, 219, 220, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 236, 238, 239, 240, 241, 244, 245, 247, 248, 249, 250, 252, 253, 254, 255, 256, 25

In [7]:
for key, val in errors.items():
    print(key, np.mean(val))

0 0.10483963647879295
1 0.18401557617471434
2 0.11641919207907689
3 0.3341352938646125
4 0.10990959525759987
6 0.1961719852427389
10 0.20660318952985887
11 0.1098854600424573
12 0.19842535294705602
13 0.13726893974916923
7 0.1119371644741481
9 0.140703936231536


In [70]:
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
mses = [x for a in errors.values() for x in a]
mses = [x**2 for x in mses]
n = len(mses)
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
BIC

-1929.8520964687173